# Import packages

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)
from scipy.stats import levy_stable

from datetime import datetime
from scipy.stats import kstest
from scipy.stats import jarque_bera
# from arch.unitroot import ADF
from scipy.stats import kurtosis
from scipy.stats import skew
# from arch import arch_model

import pickle

import ta

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import webbrowser
from datetime import datetime, timedelta

plt.style.use('classic')
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 300)

# %config inlinebackend.figure_format = 'svg'

In [ ]:
import sys

sys.path.insert(0, 'D:/Intraday_trading')

In [ ]:
from src.support import *
from src.backtest import *
from src.alphas import *

# Import and pre-processing data

## Interval data

In [ ]:
%%time

interval_data = pd.read_pickle('D:/Intraday_trading/Training/Data/XAUUSD_M15_1.pkl')

interval_data = interval_data.set_index('DATE_TIME')
interval_data.index = pd.to_datetime(interval_data.index)

interval_data['DATE'] = pd.to_datetime(interval_data['DATE'])
interval_data['OPEN'] = interval_data['OPEN']
interval_data['HIGH'] = interval_data['HIGH']
interval_data['LOW'] = interval_data['LOW']
interval_data['CLOSE'] = interval_data['CLOSE']


### Prepare_df

In [ ]:
df_4_hour = prepare_df(df = interval_data, timeframe = '4H', add_indicators = True)
df_1_day = prepare_df(df = interval_data, timeframe = '1D', add_indicators = True)
df_15_min = prepare_df(df = interval_data, timeframe = '15min', add_indicators = True)

df_1_day['WHOLE_RANGE'] = df_1_day['HIGH'] - df_1_day['LOW']
df_4_hour['WHOLE_RANGE'] = df_4_hour['HIGH'] - df_4_hour['LOW']
df_15_min['WHOLE_RANGE'] = df_15_min['HIGH'] - df_15_min['LOW']

df_1_day['GRP_WHOLE_RANGE'] = pd.qcut(df_1_day['WHOLE_RANGE'], 10)
df_4_hour['GRP_WHOLE_RANGE'] = pd.qcut(df_4_hour['WHOLE_RANGE'], 10)
df_15_min['GRP_WHOLE_RANGE'] = pd.qcut(df_15_min['WHOLE_RANGE'], 10)

df_1_day['GRP_BODY'] = pd.qcut(df_1_day['BODY'], 10)
df_4_hour['GRP_BODY'] = pd.qcut(df_4_hour['BODY'], 10)
df_15_min['GRP_BODY'] = pd.qcut(df_15_min['BODY'], 10)

df_1_day['YEAR'] = df_1_day.index.strftime('%Y')
df_1_day['WEEK'] = df_1_day.index.strftime('%Y%W')
df_1_day['MONTH'] = df_1_day.index.strftime('%Y%m')

df_4_hour['YEAR'] = df_4_hour.index.strftime('%Y')
df_4_hour['WEEK'] = df_4_hour.index.strftime('%Y%W')
df_4_hour['MONTH'] = df_4_hour.index.strftime('%Y%m')

df_15_min['YEAR'] = df_15_min.index.strftime('%Y')
df_15_min['WEEK'] = df_15_min.index.strftime('%Y%W')
df_15_min['MONTH'] = df_15_min.index.strftime('%Y%m')



In [ ]:
df_15_min.shape, df_4_hour.shape, df_1_day.shape, 

In [ ]:
df_1_day.columns

In [ ]:
# df_15_min.index[0], df_15_min.index[-1]

In [ ]:
# plot_df(df_1_day, 
#         path = None,# 'D:/Intraday_trading/Training/Saved_results/plot_df.html', 
#         open_tab = False)

# Bollinger band, RSI, Moving average

## RSI

----------------------------------------
* RSI <= 30 --> over sought --> drop?, RSI >= 70 --> over bought --> rise?; how many candle until the RSI back in the range? is there any more critical levels (50)?
* Normal parameter: 14
* RSI divergence --> Reverse

### Daily timeframe

#### Thresholds

* For 2024, 2025, only 20% of days where RSI <= 50; Daily threshold 50, 70

#### RSI = 30

* Stay under 30 for next 4 days (~70% chance)
* When recover to above 30, most likely next 3 days will be daily increase candles, for next 7 days, 50% chance that 4-5 days will be bullish candle
* Values of body/ whole range of bullish candles will be bigger --> bigger wick


In [ ]:
# rsi_summary(df_observe = df_1_day, 
#             threshold = 30, 
#             timeframe = '1D', 
#             direction_after_hit_threshold = 'UP',
#             show_distribution = True,
#             RSI_params = 14
#             )

In [ ]:
# rsi_summary(df_observe = df_1_day, 
#             threshold = 30, 
#             timeframe = '1D', 
#             direction_after_hit_threshold = 'UP',
#             show_distribution = True,
#             RSI_params = 5
#             )

#### RSI = 50

In [ ]:
# rsi_summary(df_observe = df_1_day, 
#             threshold = 50, 
#             timeframe = '1D', 
#             direction_after_hit_threshold = 'UP',
#             show_distribution = False,
#             RSI_params = 14
#             )

In [ ]:
# rsi_summary(df_observe = df_1_day, 
#             threshold = 50, 
#             timeframe = '1D', 
#             direction_after_hit_threshold = 'DOWN',
#             show_distribution = False,
#             RSI_params = 14)

In [ ]:
# rsi_summary(df_observe = df_1_day, 
#             threshold = 50, 
#             timeframe = '1D', 
#             direction_after_hit_threshold = 'UP',
#             show_distribution = False,
#             RSI_params = 5)

In [ ]:
# rsi_summary(df_observe = df_1_day, 
#             threshold = 50, 
#             timeframe = '1D', 
#             direction_after_hit_threshold = 'DOWN',
#             show_distribution = False,
#             RSI_params = 5)

#### RSI = 70

* Reversion is weak for daily RSI, num up candle when RSI reverse is skewed to the right, hard to use with daily gold

In [ ]:
# rsi_summary(df_observe = df_1_day, 
#             threshold = 70, 
#             timeframe = '1D', 
#             direction_after_hit_threshold = 'DOWN',
#             show_distribution = False,
#             RSI_params = 14)

In [ ]:
# rsi_summary(df_observe = df_1_day, 
#             threshold = 70, 
#             timeframe = '1D', 
#             direction_after_hit_threshold = 'DOWN',
#             show_distribution = False,
#             RSI_params = 5)

### 4-hour timeframe

#### RSI = 30

In [ ]:
# rsi_summary(df_observe = df_4_hour, 
#             threshold = 30, 
#             timeframe = '4H', 
#             direction_after_hit_threshold = 'UP',
#             show_distribution = False,
#             RSI_params = 14)

In [ ]:
# rsi_summary(df_observe = df_4_hour, 
#             threshold = 30, 
#             timeframe = '4H', 
#             direction_after_hit_threshold = 'UP',
#             show_distribution = False,
#             RSI_params = 5)

In [ ]:
# rsi_summary(df_observe = df_4_hour[df_4_hour.index >= datetime(2024, 1, 1)], 
#             threshold = 30, 
#             timeframe = '4H', 
#             direction_after_hit_threshold = 'UP',
#             show_distribution = False,
#             RSI_params = 14)

In [ ]:
# rsi_summary(df_observe = df_4_hour[df_4_hour.index >= datetime(2024, 1, 1)], 
#             threshold = 30, 
#             timeframe = '4H', 
#             direction_after_hit_threshold = 'UP',
#             show_distribution = False,
#             RSI_params = 5)

#### RSI = 70

In [ ]:
# rsi_summary(df_observe = df_4_hour, 
#             threshold = 70, 
#             timeframe = '4H', 
#             direction_after_hit_threshold = 'DOWN',
#             show_distribution = False,
#             RSI_params = 14)

In [ ]:
# rsi_summary(df_observe = df_4_hour, 
#             threshold = 70, 
#             timeframe = '4H', 
#             direction_after_hit_threshold = 'DOWN',
#             show_distribution = False,
#             RSI_params = 5)

In [ ]:
# rsi_summary(df_observe = df_4_hour[df_4_hour.index >= datetime(2024, 1, 1)], 
#             threshold = 70, 
#             timeframe = '4H', 
#             direction_after_hit_threshold = 'DOWN',
#             show_distribution = False,
#             RSI_params = 14)

In [ ]:
# rsi_summary(df_observe = df_4_hour[df_4_hour.index >= datetime(2024, 1, 1)], 
#             threshold = 70, 
#             timeframe = '4H', 
#             direction_after_hit_threshold = 'DOWN',
#             show_distribution = False,
#             RSI_params = 5)

### 15-min timeframe

#### RSI = 30

In [ ]:
# rsi_summary(df_observe = df_15_min[df_15_min.index >= datetime(2024, 1, 1)], 
#             threshold = 30, 
#             timeframe = '15min', 
#             direction_after_hit_threshold = 'UP',
#             show_distribution = False,
#             RSI_params = 5)

#### RSI = 70

In [ ]:
# rsi_summary(df_observe = df_15_min[df_15_min.index >= datetime(2024, 1, 1)], 
#             threshold = 70, 
#             timeframe = '15min', 
#             direction_after_hit_threshold = 'DOWN',
#             show_distribution = False,
#             RSI_params = 5)

## Moving average

----------------------------------------
* Price >= MA --> Rise, Price <= MA --> drop
* Low MA cut and go above high MA --> rise, High MA go below low MA --> drop
* Act as support, resistance
* Normal parameter: 50, 200, exponential

### 4-hour time frame

In [ ]:
# ema_summary(df_observe = df_4_hour.copy(), EMA_params = [10, 50, 200], timeframe = '4H')


### 15-min time frame

In [ ]:
# ema_summary(df_observe = df_15_min.copy(), EMA_params = [10, 50, 200], timeframe = '15min')


## Bollinger band


* BB expands --> Continue big trend; BB narrows --> Accumulation
* Touch or out from upper/ lower band --> reverse/ continue/ accumulation and continue/ accumulation and reverse? <-> Act as support and resistance
* Normal parameter: (50, 2)


In [ ]:
# bollingerband_summary(df_observe = df_15_min.copy(), timeframe = '15min', mean = 20, std = 2)